# MTH 651: Advanced Numerical Analysis

## Homework Assignment 4

### (Student Name 👈 Write Your Name Here!)

### Guidelines

* Each student must complete their own assignment individually.
  * Discussing with other students is allowed (encouraged!), but you must write your own answers and not copy off of others.
* Submit the homework as a Jupyter notebook with **properly formatted LaTeX**


#### Problem 1. (2 points)

Let $A$ be a nonsingular square matrix.
Prove that $A^T A$ is symmetric and positive-definite.

#### Problem 2a. (2 points)

Let $A \in \mathbb{R}^{n \times n}$ be SPD, let $b \in \mathbb{R}^n$ be given.
Define the quadratic form
$$
    Q(z) = z^T A z - 2 z^T b.
$$
Prove that $x = A^{-1}b$ minimizes $Q$.

#### Problem 2b. (1 point)

How does this relate to remark 2.5.11 in the textbook?

#### Problem 3. (3 points)

Let $A$ be SPD.
Prove that
$$
    (x, y)_A = x^T A y
$$
defines an inner product (this is known as the $A$-inner product).

Write down the definition of the induced norm $\| \cdot \|_A$.

#### Problem 4.

Recall that the Gauss-Seidel method can be obtained by considering the **matrix splitting**
$$
    A = L + U
$$
(where $L$ is lower-triangular, and $U$ is strictly upper triangular).
Then, the system
$$
    A x = b
$$
becomes
$$
    L x + U x = b,
$$
and lagging the $U$ term, we obtain an iteration:
$$
    L x^{(i+1)} + U x^{(i)} = b,
$$
equivalently
$$
    x^{(i+1)} = L^{-1}( b - U x^{(i)})
$$

---

We define the **Jacobi** method using the splitting
$$
    A = L + U + D,
$$
where $L$ and $U$ are **strictly** lower and upper triangular, and $D = \operatorname{diag}(A)$.
The Jacobi method lags the terms associated with $L$ and $U$, obtaining the iteration
$$
    x^{(i+1)} = D^{-1}(b - (L+U)x^{(i)})
$$

#### Problem 4a. (2 points)

Show that this iteration is equivalent to
$$
    x^{(i+1)} = x^{(i)} + D^{-1} ( b - Ax^{(i)})
$$
The quantity $r^{(i)} = b - Ax^{(i)}$ is known as the **residual**.

#### Problem 4b. (2 points)

The **error** $e^{(i)} = x - x^{(i)}$ is the error.
Show that $r^{(i)} = A e^{(i)}$ and that the error satisfies the recurrence
$$
    e^{(i+1)} = (I - D^{-1}A)e^{(i)}.
$$

#### Problem 4c. (2 points)

Let $G = I - D^{-1}A$ denote the Jacobi iteration matrix.
Write a formula for the error $e^{(i)}$ in terms of $G$, the initial guess $x^{(0)}$, and the true solution $x$.

#### Problem 4d. (2 points)

A matrix $A = (a_{ij})$ is called **diagonally dominant** if
$$
    | a_{ii} | \geq \sum_{j \neq i} | a_{ij} |
$$
for all $i$.
$A$ is called **strictly diagonally dominant** if the inequality is strict.

Prove that the Jacobi method is guaranteed to converge when $A$ is strictly diagonally dominant.

(Hint: consider the $\ell_\infty$ norm of the iteration matrix $G$).



#### Problem 5. (Bonus)

How does the Jacobi method differ from the Gauss-Seidel method from the point of view of parallel computing?

#### Problem 6. (Coding)

Write a program using NumPy and SciPy to implement the Gauss-Seidel method. (Use NumPy linear algebra operations instead of explicit loops).

The following functions may be useful:

* `np.tril` returns the lower triangular part of a matrix (including diagonal)
* `scipy.linalg.solve_triangular` solves a triangular linear system (using backwards/forwards substitution)
    * In particular, it will be useful to use `solve_triangular(L, v, lower=True)`
* `np.linalg.norm` returns the norm of a vector
* If `A` is a matrix, the matrix-vector product $Ax$ is written `A.dot(x)`

Use the following code to print information at every iteration:
```py
    # i is the iteration number, resnorm is the norm of the residual vector
    print("{:5d}             {:.8e}".format(i, resnorm))
```

In [ ]:
import numpy as np
from scipy.linalg import solve_triangular

def gauss_seidel(A, b, maxit, tol):
    """
    gauss_sidel(A, b, maxit, tol)

    Solves the system Ax = b using the Gauss-Seidel method with zero initial
    guess.

    The iteration will terminate after at most maxit iterations.

    If the norm of the residual vector is less than tol, the iteration will
    terminate early.

    At every iteration, print the iteration number and the norm of the residual.
    """


Solve the following system using your implementation of Gauss-Seidel.

* Is Gauss-Seidel gauaranteed to converge for all such random matrices? Why or why not?

In [ ]:
n = 20
A = np.random.rand(n, n)
A = A.transpose().dot(A)
alpha = 10.0
A += alpha*np.diag(np.diag(A))
b = np.random.rand(n)

gauss_seidel(A, b, 1000, 1e-8);


In [ ]:
def jacobi(A, b, maxit, tol):
    """
    jacobi(A, b, maxit, tol)

    Solves the system Ax = b using the Jacobi method with zero initial guess.

    The iteration will terminate after at most maxit iterations.

    If the norm of the residual vector is less than tol, the iteration will
    terminate early.

    At every iteration, print the iteration number and the norm of the residual.
    """


Solve the following system using your implementation of Gauss-Seidel.

* Is Jacobi gauaranteed to converge for all such random matrices? Why or why not?
* What role does the parameter `alpha` play?
* What happens when $\alpha < 1$?
* What happens with $\alpha \gg 1$ ($\alpha = 10^3, \alpha = 10^6$, etc.)?

In [ ]:
n = 20
A = np.random.rand(n, n)
alpha = 1.5
A = A + alpha*np.diag(np.sum(np.abs(A - np.diag(np.diag(A))), axis=1))

xi = np.zeros(n)

b = np.random.rand(n)
jacobi(A, b, 1000, 1e-8);
